![CH8-ADS.png](.\Media\CH8-ADS.png)
# <span style="color:#cc5500;">Deploy Azure SQL VM from the Azure Portal </span>

There are several ways to deploy a net-new SQL Server to Azure VM's.  In the following notebooks we will cover most scenarios on how you would deploy SQL Server to the cloud:

- Deploy Azure SQL VM's using the Azure Portal - Covered in this notebook
- Deploy Azure SQL VM's using Power Shell - Covered in the next notebook
- Deploy Azure SQL VM's using an ARM Template - Covered in a later notebook

The 3 bullets above are for a net-new SQL Server on Azure VM.  The simplist form to just get you used to using Azure and seeing how this work.  In a later notebook we will do more of a make-it-real-scenario and show you how to migrate an actual on-premises production SQL Server to Azure.

  

At this time, I do not plan to cover how to deploy an Azure VM from a blank Operating System, configure all proprietary software and then how to upload SQL Server Media and to a clean install of SQL on the VM's.  That process is very lengthy and for the purpose of this book, while still be covering a tremendous amount of SQL Server subjects, but at the same time, keeping it simple when possible.

  

It is assumed that you already have either a Paid subscription or a Trial Subscription.  If you do not, you can go to [Create Your Azure Free Account Today | Microsoft Azure](https://azure.microsoft.com/en-us/free/)

  

Login to the Commercial Azure Portal here:  https://portal.azure.com/  

- Everyone reading this notebook that has an Azure Subscription, has access to Azure Commercial

  

As an alternative, I will be using Microsoft Azure Government by logging in to:

- For those members that have a subscription in an Azure Government GCC Tenant, you can connect to [Home - Microsoft Azure Government](https://portal.azure.us/#home)
- Many of my samples will show Azure Government GCC because there are generally a few more steps for you to take to get our tools like SSMS, Visual Studio Code, Azure Data Studio to connect to Government Clouds, and I want to demonstrate that for you when possible.

## <span style="color:#cc5500;">Choose a Market Place Image</span>

Use the Azure Marketplace to choose one of several pre-configured images from the virtual machine gallery.  Again, to keep this as simple as possible, we are going to use a image already in the Gallery. 

The Developer edition is used in this notebook because it is a full-featured, free edition of SQL Server for development testing.  You pay only for the cost of running the VM.  However, you are free to choose any of the images to use in this walkthrough. For a description of available images, see this link: [Overview of SQL Server on Azure Windows Virtual Machines - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/sql-server-on-azure-vm-iaas-what-is-overview?view=azuresql#payasyougo)

Licensing costs for SQL Server are incorporated into the per-second pricing of the VM you create and varies by edition and cores. However, SQL Server Developer edition is free for development and testing, not production. Also, SQL Express is free for lightweight workloads (less than 1 GB of memory, less than 10 GB of storage). You can also bring-your-own-license (BYOL) and pay only for the VM. Those image names are prefixed with {BYOL}. For more information on pricing see this link: [Price guidance & managing costs - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/pricing-guidance?view=azuresql)

To choose an image, follow these steps:

1. From the home page of the Portal click the Marketplace Icon, or in the search bar enter "Marketplace" and then click the icon when it appears.  The UI of the Portal changes often, so your view may be slightly different.  Click Create Virutal Machine

    
    ![CH8-Gallery.jpg](.\Media\CH8-Gallery.jpg)
    
2. Select Azure SQL in the left-hand menu of the Azure portal. If Azure SQL is not in the list, select All services, then type Azure SQL in the search box. You can select the star next to Azure SQL to save it as a favorite to pin it to the left-hand navigation.

    
    ![CH8-ChooseImage.jpg](.\Media\CH8-ChooseImage.jpg)

    
3. Select + Create to open the Select SQL deployment option page. Select the Image drop-down and then select an image.  I choose Free SQL Server License: SQL Server 2019 on wondws Server 2022.  You can also choose Show details for additional information about the image.

## <span style="color:#cc5500;">Basic Settings</span>

The Basics tab allows you to select the subscription, resource group, and instance details.

Using a new resource group is helpful if you are just testing or learning about SQL Server deployments in Azure.  After you finish with your test, delete the resource group to automatically delete the VM and all resources associated with that resource group. For more information about resource groups, see Azure Resource Manager Overview.

On the Basics tab, provide the following information:

- Under Project Details, make sure the correct subscription is selected.
- In the Resource group section, either select an existing resource group from the list or choose Create new to create a new resource group. A resource group is a collection of related resources in Azure (virtual machines, storage accounts, virtual networks, etc.).

Under Instance details:

- Enter a unique Virtual machine name.
- Choose a location for your Region.
- For the purpose of this guide, leave Availability options set to No infrastructure redundancy required. To find out more information about availability options, see Availability.
- In the Image list, select Free SQL Server License: SQL Server 2019 Developer on Windows Server 2019 if it's not already selected.
- Choose Standard for Security type.
- Select See all sizes for the Size of the virtual machine and choose a size.  I choose a small one for this demo.  If this is for testing purposes, be sure to clean up your resources once you're done with them to prevent any unexpected charges
- Under Inbound port rules, choose Allow selected ports and then select RDP (3389) from the drop-down so that you can RDP to the server after it has been created

![CH8-Basics.jpg](.\Media\CH8-Basics.jpg)

## <span style="color:#cc5500;">Discs</span>

On the Disks tab, configure your disk options.

- Under OS disk type, select the type of disk you want for your OS from the drop-down. Premium is recommended for production systems but is not available for a Basic VM. To use a Premium SSD, change the virtual machine size. For the purpose of this demo, I choose Standard SSD
- Under Advanced, select Yes under use Managed Disks.

Microsoft recommends Managed Disks for SQL Server. Managed Disks handles storage behind the scenes. In addition, when virtual machines with Managed Disks are in the same availability set, Azure distributes the storage resources to provide appropriate redundancy. For more information, see Azure Managed Disks Overview. For specifics about managed disks in an availability set, see Use managed disks for VMs in availability set.

## <span style="color:#cc5500;">Networking</span>

On the Networking tab, configure your networking options.

- Create a new virtual network or use an existing virtual network for your SQL Server VM. Designate a Subnet as well.
- Under NIC network security group, select either a basic security group or the advanced security group. Choosing the basic option allows you to select inbound ports for the SQL Server VM which are the same values configured on the Basic tab. Selecting the advanced option allows you to choose an existing network security group, or create a new one.
- You can make other changes to network settings, or keep the default values.  For this demo, I just use the default values

## <span style="color:#cc5500;">Management</span>

On the Management tab, configure monitoring and auto-shutdown.

- Azure enables Boot diagnostics by default with the same storage account designated for the VM. On this tab, you can change these settings and enable OS guest diagnostics.
- You can also enable System assigned managed identity and auto-shutdown on this tab.
- For this demo, I select my timezone and accept all remaining default options
- I Skip the Advanced tab and go directly to the SQL Server Settings Tab next

## <span style="color:#cc5500;">SQL Server settings</span>

On the SQL Server settings tab, configure specific settings and optimizations for SQL Server. You can configure the following settings for SQL Server:

- Connectivity
- Authentication
- Azure Key Vault integration
- Storage configuration
- SQL instance settings
- Automated patching
- Automated backup
- Machine Learning Services